<a href="https://colab.research.google.com/github/bhargavpirates/TwitterAPI-Sentimental-Analysis/blob/master/TwitteAPii_Sentimental_Analysis_with_MultiDomain_as_Training_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## Sentiment analysis on DomainSentiment Data


### import Libraries

In [0]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns

import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import pickle

from tqdm import tqdm
import os


from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

#from gensim.models import Word2Vec
#from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm


from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

### working this problem on googleCollab So mounting Google DRive data into Collab

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!cp -r  "/content/drive/My Drive/domain_sentiment_data.tar/sorted_data_acl/books" "books"
!cp -r  "/content/drive/My Drive/domain_sentiment_data.tar/sorted_data_acl/dvd" "dvd"
!cp -r  "/content/drive/My Drive/domain_sentiment_data.tar/sorted_data_acl/electronics" "electronics"
!cp -r  "/content/drive/My Drive/domain_sentiment_data.tar/sorted_data_acl/kitchen_&_housewares" "kitchen_&_housewares"

## From XML Data extracting requried ReviewText Data and storing them in a .txt file

In [0]:
data_list=["dvd/","books/","electronics/","kitchen_&_housewares/"]
for idx in range(4):

    pos_file = data_list[idx] +"positive.review"
    neg_file = data_list[idx] +"negative.review"
   
    with open(pos_file,'r',encoding='utf-8') as f:
        pos = f.readlines()
    with open(neg_file,'r',encoding='utf-8') as f:
        neg = f.readlines()
    
    lst ,final_pos,final_neg=[],[],[]
    
    for i in range(len(pos)):
        if(pos[i]=="<review_text>\n"):lst.append(i)
        elif(pos[i]=="</review_text>\n"):lst.append(i)   
        if(len(lst)==2):
            a=pos[ lst[0]+1 : lst[1] ]
            stng=" ".join(a)
            lst.clear()
            final_pos.append(stng.replace("\n",""))
            
    for i in range(len(neg)):
        if(neg[i]=="<review_text>\n"):lst.append(i)
        elif(neg[i]=="</review_text>\n"):lst.append(i)
        if(len(lst)==2):
            a=neg[ lst[0]+1 : lst[1] ]
            stng=" ".join(a)
            lst.clear()
            final_neg.append(stng.replace("\n",""))
     
    
    train , test = [], []  
    train=final_pos[:int(len(final_pos)*0.8)] + final_neg[:int(len(final_neg)*0.8)]
    test=final_pos[int(len(final_pos)*0.8):] + final_neg[int(len(final_neg)*0.8):]
    
    train_ylabel = [1 for i in range(int(len(final_pos)*0.8))] + [0 for i in range(int(len(final_pos)*0.8))]
    
    train_file = data_list[idx]+"trainnew.txt" 
    test_file = data_list[idx]+"testnew.txt"
    
    with open(train_file,'w',encoding='utf-8') as f:
        for i in range(len(train)):
            f.write(train[i])
            f.write("\n")
            
    with open(test_file,'w',encoding='utf-8') as f:
        for i in range(len(test)):
            f.write(test[i])
            f.write("\n")

### importing nltk library
*  for stopwords
* Lemmatizer

In [0]:
nltk.download('stopwords')
nltk.download('wordnet')

In [0]:
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()    

stopwords=set(stopwords.words('english'))        
no_stopwords=["against","aren't","couldn't",'didn',"didn't", "doesn't", "don't", 'few', "hadn't", "hasn't", "haven't","isn't" ,'just',"mightn't",'more','most',
 "mustn't","no","nor","not","needn't","once","out","wasn't","weren't", "won't" , 'won', "wouldn't",'why','any','only','very']
modified_stopwords=list(set(stopwords) - set(no_stopwords))

## Preprocessing Data

In [0]:
import re

#reading file
def preprocess(filename):
    lst=[]
    print(filename)
    
    for file in filename:
      with open(file ,"r") as f:
        for line in f.read().split('\n'):
          lst.append(line)
          
    #print("length of lst  :::: {} ".format(len(lst)))
    lst = [ i.replace("aren't","are not").replace("couldn't","could not").replace("wasn't","was not") for i in lst ]
    lst = [ i.replace("weren't","were not").replace("mustn't","must not").replace("won't","not").replace("wouldn't","would not") for i in lst ]
    lst = [re.sub(r'[^a-zA-Z ]',r'' , (" ".join([lemmatizer.lemmatize(i.lower()) for i in line.split()  if i not in modified_stopwords])) ) for line in lst ]
    lst = [line for line in lst if line!='']
    #lst = lst[:-1]
    
    return lst
    

x_train=preprocess(["dvd/trainnew.txt","books/trainnew.txt","electronics/trainnew.txt","kitchen_&_housewares/trainnew.txt"])
x_test=preprocess(["dvd/testnew.txt","books/testnew.txt","electronics/testnew.txt","kitchen_&_housewares/testnew.txt"])

y_train=[]
y_train= [1 if(i<800) else 0 for i in range(1600)]*4
y_test= [1 if(i<200) else 0 for i in range(400)]*4

print("length of x_train ::: {}".format(len(x_train)))
print("length of x_test ::: {}".format(len(x_test)))
print("length of y_train ::: {}".format(len(y_train)))
print("length of y_test ::: {}".format(len(y_test)))

## Coverting Text Data into Vector Using TFIDF Vectorizer

In [0]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,3), min_df=10, max_df=0.95,stop_words='english',max_features=50000 )
tf_idf_vect.fit(x_train)
print("some sample features(unique words in the corpus)",tf_idf_vect.get_feature_names()[0:10])
print('='*50)

X_train_tfidf= tf_idf_vect.transform(x_train)
print("the type of count vectorizer ",type(X_train_tfidf))
print("the shape of out text TFIDF vectorizer ",X_train_tfidf.get_shape())
print("the number of unique words including both unigrams and bigrams ", X_train_tfidf.get_shape()[1])

X_test_tfidf = tf_idf_vect.transform(x_test)
print("the type of count vectorizer ",type(X_test_tfidf))
print("the shape of out text TFIDF vectorizer ",X_test_tfidf.get_shape())
print("the number of unique words ", X_test_tfidf.get_shape()[1])


### Applying Standard Scalar

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=False)
scaler.fit(X_train_tfidf)

X_train_tfidf=scaler.transform(X_train_tfidf)
X_test_tfidf=scaler.transform(X_test_tfidf)

## MODEL 1 :: Applying LogisticRegression using penalty ='L2'

### Finding Best HyperParaneter

In [0]:
import math
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

#algoname=LogisticRegression
#param_grid={'C':[1000,100,10,1,0.1,0.001,0.0001,0.00001]}
def algo(algoname,param_grid,plot=True):
  clf=algoname()

  gcv=GridSearchCV(clf,param_grid,cv=5,scoring='roc_auc')
  gcv.fit(X_train_tfidf,y_train)
  print(gcv.best_params_)
  print(gcv.best_score_)
  
  val=list(param_grid.keys())
  print(val)
  hyper_parameters=gcv.get_params()['param_grid'][val[0]]
  test_scores=gcv.cv_results_['mean_test_score'].tolist()

  log_hyper_parameters=[math.log(i) for i in hyper_parameters]
  if(plot):
    plt.plot( log_hyper_parameters ,test_scores , color='green', linestyle='dashed', linewidth = 3,marker='o', markerfacecolor='blue', markersize=12,  label='Test plot')
    plt.xlabel("hyper_parameters (C)")
    plt.ylabel("Model performance")
    plt.legend()
  
  print(list(gcv.best_params_.values()))
  
  return list(gcv.best_params_.values())[0]
  #print(gcv)

#### https://stackoverflow.com/questions/22851316/what-is-the-inverse-of-regularization-strength-in-logistic-regression-how-shoul

### Finding Metrics values , ploting ROC_AUC Curve and Confuion MAtrix

In [0]:

def metric(algoname,best_hyperparameter):
  #clf1=algoname(C=best_hyperparameter)
  #print(param)
  if(algoname==LogisticRegression):
    clf1=algoname(C=best_hyperparameter)
  else:
    clf1=algoname(best_hyperparameter)
  clf1.fit(X_train_tfidf,y_train)
  pred_train=clf1.predict(X_train_tfidf)
  pred=clf1.predict(X_test_tfidf)

  print(pred)
  fpr_train,tpr_train,thresholds_train=roc_curve(y_train,pred_train)
  print("AUC Score for train data  :",metrics.auc(fpr_train,tpr_train))
  fpr,tpr,thresholds=roc_curve(y_test,pred)
  print("AUC Score for test data   :",metrics.auc(fpr,tpr))
  print("Accuracy Score            : ",accuracy_score(y_test,pred)*100)
  print("Precision Score           : ",precision_score(y_test,pred)*100)
  print("Recall Score              : ",recall_score(y_test,pred)*100)
  print("F1 Score                  : ",f1_score(y_test,pred)*100)

  confusionmatrix_DF=pd.DataFrame(confusion_matrix(y_test,pred),columns=['0','1'],index=['0','1'])
  sns.heatmap(confusionmatrix_DF,annot=True,fmt='g',cmap='viridis')
  plt.title("Confusion matrix ")
  plt.show()
  
  return clf1

In [0]:
algoname=LogisticRegression
#algoname=LogisticRegression
param_grid={'C':[1000,100,10,1,0.1,0.001,0.0001,0.00001]}
best_hyperparameter= algo(algoname,param_grid,False)
print(best_hyperparameter)
clf1 = metric(algoname,best_hyperparameter)

## Important Features on TFIDF

In [0]:
feature_names=tf_idf_vect.get_feature_names()
coefs=sorted(zip(clf1.coef_[0],feature_names))

top20Negative=coefs[:5]
top20Postive=coefs[::-1][:5]

res_neg=pd.DataFrame(top20Negative,columns=['values','NegativeFeatures'])
res_pos=pd.DataFrame(top20Postive,columns=['values','PostiveFeatures'])
pd.concat([res_neg,res_pos],axis=1)

## Predicting unlabeled Reviews

In [0]:
data_list=["dvd/","electronics/","kitchen_&_housewares/"]
final_unlabeled=[]
for idx in range(3):
  unlabeled_file = data_list[idx] +"unlabeled.review"
  with open(unlabeled_file,'r',encoding='utf-8') as f:
    unlabeled = f.readlines()
  lst = []
  for i in range(len(unlabeled)):
    if(unlabeled[i]=="<review_text>\n"):lst.append(i)
    elif(unlabeled[i]=="</review_text>\n"):lst.append(i)   
    if(len(lst)==2):
      a=unlabeled[ lst[0]+1 : lst[1] ]
      lst.clear()
      final_unlabeled.append(" ".join(a).replace("\n",""))
  print("total number of rows in final_unlabeled file :::: {}".format(len(final_unlabeled)))


In [0]:
final_unlabeled = [ i.replace("aren't","are not").replace("couldn't","could not").replace("wasn't","was not") for i in final_unlabeled ]
final_unlabeled = [ i.replace("weren't","were not").replace("mustn't","must not").replace("won't","not").replace("wouldn't","would not") for i in final_unlabeled ]
final_unlabeled = [re.sub(r'[^a-zA-Z ]',r'' , (" ".join([lemmatizer.lemmatize(i.lower()) for i in line.split()  if i not in modified_stopwords])) ) for line in final_unlabeled ]
final_unlabeled = [line for line in final_unlabeled if line!='']

In [0]:
unlabeled_tfidf          =  tf_idf_vect.transform(final_unlabeled)
scalar_unlabeled_tfidf   =  scaler.transform(unlabeled_tfidf)
unlabel_predicted        =  clf1.predict(scalar_unlabeled_tfidf)

 ### Model2  :: MultinomialNB

In [0]:
from  sklearn.naive_bayes import MultinomialNB
algoname=MultinomialNB
#alpha = [0.00001, 0.0001, 0.001, 0.1, 1, 10, 100,1000]
param_grid={'alpha':[1000,100,10,1,0.1,0.001,0.0001,0.00001]}
best_hyperparameter=algo(algoname,param_grid)

In [0]:
#algoname=LogisticRegression
#best_hyperparameter= algo(algoname,param_grid,False)
#clf1 = metric(algoname,best_hyperparameter)

best_hyperparameter= algo(algoname,param_grid,False)
print(best_hyperparameter)
clf1 = metric(algoname,best_hyperparameter)

### Remaning Models


In [0]:
!pip install xgboost
from xgboost import XGBClassifier

In [0]:
import xgboost as xgb
#dtrain = xgb.DMatrix(X_train_xg.values, label=y_train_xg.values)
#dtest = xgb.DMatrix(X_test_xg.values, label=y_test_xg.values)
param_test1 = {
'max_depth': [5,10,12,16,20],
'min_child_weight':[2,4,6,8],
'reg_alpha':[0.00001,0.001,0.01,1]
}

gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate =
0.1, n_estimators=140, max_depth=5,
min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27),
param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train_tfidf,y_train)
#gsearch1.cv_results_
gsearch1.best_params_, gsearch1.best_score_

In [0]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics.classification import accuracy_score, log_loss
clf1=xgb.XGBClassifier(min_child_weight=2,max_depth=16 , reg_alpha=0.01)
clf1.fit(X_train_tfidf,y_train)
pred_train=clf1.predict(X_train_tfidf)
pred=clf1.predict(X_test_tfidf)
print(pred)


print("Accuracy Score : ",accuracy_score(y_test,pred)*100)
print("Precision Score : ",precision_score(y_test,pred)*100)
print("Recall Score : ",recall_score(y_test,pred)*100)
print("F1 Score : ",f1_score(y_test,pred)*100)

# Example:: Twitter


In [0]:
! python -m pip install tweepy

In [0]:
import tweepy
import re

from tweepy import OAuthHandler

consumer_api_key = 'ED7DTkkIg7bkAsrUt3Lj5kSM6'
consumer_api_secret = 'TJA4rA7nRVhO5dmGZnQPhdegPyqgSuGt4exxyCcCnsqJXi6WFT' 
access_token = '516506229-qdHN2J6EACuFvuyAXisB0Pe4zeJykOzufzieesxu'
access_token_secret ='q8eig46jC4giFe6AFn8VnqkM32adW9uccLxVROpzbp2Ap'


authorizer = OAuthHandler(consumer_api_key, consumer_api_secret)
authorizer.set_access_token(access_token, access_token_secret)

In [0]:
#api = tweepy.API(authorizer ,timeout=15)
api = tweepy.API(authorizer, wait_on_rate_limit=True)

all_tweets = []

search_query = 'IsroPerGarvHai'

for tweet_object in tweepy.Cursor(api.search,q=search_query+" -filter:retweets",lang='en',result_type='recent').items():
    all_tweets.append(tweet_object.text)

In [0]:
all_tweets = ["".join(re.sub(r'https?:\/\/.*[\r\n]*','', i )) for i in all_tweets]
all_tweets  = ["".join(re.sub(r'[^a-zA-Z0-9 ]','',i)) for i in all_tweets]
tweet_scrap = [ i.replace("aren't","are not").replace("couldn't","could not").replace("wasn't","was not") for i in all_tweets ]
tweet_scrap = [ i.replace("weren't","were not").replace("mustn't","must not").replace("won't","not").replace("wouldn't","would not") for i in all_tweets ]
tweet_scrap = [re.sub(r'[^a-zA-Z ]',r'' , (" ".join([lemmatizer.lemmatize(i.lower()) for i in line.split()  if i not in modified_stopwords])) ) for line in all_tweets ]
tweet_data = [line for line in tweet_scrap if line!='']

In [0]:
tweet_tfidf     =  tf_idf_vect.transform(tweet_data)
scalar_tweet_tfidf     =  scaler.transform(tweet_tfidf)
final_tweet_tfidf  =  clf1.predict(scalar_tweet_tfidf)

In [0]:
final_tweet_tfidf

In [0]:
np_tweet_data = np.array(tweet_data)
pd.DataFrame({"Tweet_Data" : np_tweet_data,"label" :final_tweet_tfidf})